<a href="https://colab.research.google.com/github/tdhoang0111/Coursera-test/blob/main/8_data_aggregation_ph_n_t_ch_d_li_u_v_i_python_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VIII. DATA AGGREGATION & GROUP OPERATIONS

Phân loại dữ liệu (categorizing) và áp dụng các function trên mỗi nhóm, cho dù là tổng hợp hay biến đổi dữ liệu thường là một thành phần quan trọng của quá trình phân tích dữ liệu. Sau khi tải (đọc) dữ liệu, gộp, và chuẩn bị dữ liệu, bạn sẽ muốn tính toán thông kê theo các nhóm hoặc tạo pivot table sử dụng cho mục đích trực quan hóa dữ liệu (data visualization). Pandas cung cấp giao diện `groupby` cho phép cắt, đảo và tóm tắt dataset theo một cách tự nhiên nhất.

Một lý do cho sự phổ biến của các loại cơ sở dữ liệu quan hệ và SQL là sự đơn giản trong việc thêm, lọc hay chuyển đổi và tổng hợp dữ liệu. Tuy nhiên các ngôn ngữ truy vấn như SQL bị hạn chế trong các thao tác xử lý dữ liệu theo nhóm mà chúng có thể thực hiện được. Với Python và Pandas, bạn có thể thực hiện bất kỳ thao tác nào đối với các nhóm dù là khá phức tạp bằng cách sử dụng bất kỳ hàm nào chấp nhận đối tượng Pandas hoặc Numpy array.

## 8.1. GroupBy Mechanics

Hadley Wickham, một tác giả của nhiều package phần mềm phổ biến cho ngôn ngữ lập trình R, đã đặt ra thuật ngữ split-apply-merge để mô tả các thao tác với nhóm. Trong giai đoạn đầu của việc xử lý dữ liệu, dữ liệu được lưu trữ dưới dạng các pandas object, dù là Series hay DataFrame thì đều được chia ra thành các nhóm dựa trên 1 hoặc nhiều key mà bạn cung cấp. Việc phân tách dữ liệu (split) được thực hiện trên 1 trục cụ thể của đối tượng. Ví dụ, DataFrame có thể đưuợc nhóm dựa trên các hàng (axis=0) hoặc cột (axis=1). Khi việc này được thực hiện, 1 function được áp dụng cho mỗi nhóm, tạo ra 1 giá trị mới. Cuối cùng, kết quả của việc áp các function này được kết hợp thành 1 đối tượng kết quả. Hình thức của các đối tượng kết quả thường sẽ phụ thuộc vào những gì đang thực hiện với dữ liệu. Hình dưới đây mô tả mô hình này:


![](https://thinhvu.com/wp-content/uploads/python_course/groupby_group_split_apply.png)

Mỗi key của nhóm có thể có nhiều dạng, và các key này không nhất thiết phải cùng 1 loại:

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.548810,0.444728
1,a,two,1.950637,-1.690023
2,b,one,-0.272943,-1.925146
3,b,two,-0.308027,0.921635
4,a,one,1.720689,-1.169202


Giả sử bạn muốn tính mean của cột `data1`, sử dụng label từ cột `key1`. Có nhiều cách để thực hiện được điều này. Một trong số đó là truy cập `data1` và gọi `groupby` method với cột `key1`:

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

Biến `grouped` giờ đây trở thành 1 đối tượng `GroupBy`. Thực tế chưa có phép tính toán nào được thực hiện bởi đối tượng `GroupBy` ngoại trừ 1 số dữ liệu trung gian về group key `['key1']`. Ý tưởng ở đây là đối tượng này chứa tất cả thông tin cần thiết để sau đó áp dụng 1 số thao tác tính toán trên mỗi nhóm. Ví dụ: để tính toán mean cho nhóm chúng ta có thể làm như sau:

In [ ]:
grouped.mean()

key1
a    0.428911
b   -1.001065
Name: data1, dtype: float64

Ở đây, dữ liệu được tổng hợp theo group key, tạo ra 1 Series mới được index bởi giá trị unique trong cột `key1`. Index của kết quả có tên `key1` bởi vì nó được tạo ra bởi giá trị của cột `df['key1']` trong DataFrame. Nếu chúng ta truyền 1 list là các cột của DataFrame vào `groupby` method, chúng ta sẽ có kết quả như sau:

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.786286
      two    -0.285840
b     one    -0.382953
      two    -1.619178
Name: data1, dtype: float64

Trên đây chúng ta group dữ liệu sử dụng 2 key và Series kết quả bây giờ sẽ có index phân cấp bao gồm cặp key được quan sát:

In [ ]:
means.unstack()

key2,one,two
key1,,
a,0.786286,-0.285840
b,-0.382953,-1.619178


Ở ví dụ dưới đây, group key là 1 Series

In [ ]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.285840
            2006   -0.382953
Ohio        2005   -0.568762
            2006    1.090917
Name: data1, dtype: float64

Thông thường, thông tin cần được sử đụng dể tạo group sẽ có sẵn trong DataFrame. Ví dụ dưới đây, bạn có thể truyền vào tên các cột như là group key:

In [ ]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.428911,0.142026
b,-1.001065,-0.725598


In [ ]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.786286 -0.704666
     two  -0.285840  1.835409
b    one  -0.382953 -0.669161
     two  -1.619178 -0.782035

Nếu để ý, bạn có thể nhận thấy trong trường hợp đầu tiên df.groupby('key1').mean() rằng không thấy sự xuất hiện của cột `key2` trong kết quả của groupby method. Lý do là vì df['key2'] không phải là dữ liệu dạng số vì thế bị loại ra khỏi kết quả tính toán. Mặc định, tất cả các cột dữ liệu dạng số đều sẽ được tổng hợp, dù vậy, bạn có thể lọc 1 subset để chỉ rõ dữ liệu nào cần được tổng hợp.

Bất kể mục đích của việc sử dụng `groupby` là gì, `size` method là 1 cách hữu ích để trả về 1 Series chứa thông tin của group size (số phần tử trong group).

In [ ]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

> <font color='blue'>Lưu ý: Tất cả missing value trong group key sẽ bị loại bỏ khỏi kết quả tính toán.</font>

### Iterating Over Groups

`GroupBy` object hỗ trợ việc lặp (iteration), tạo ra 1 chuỗi 2 tupple chứa group name và đoạn dữ liệu. Cùng xem ví dụ sau:

In [ ]:
for name, group in df.groupby('key1'):
  print(name)
  print(group)

a
  key1 key2     data1     data2
0    a  one  0.481654 -1.107212
1    a  two -0.285840  1.835409
4    a  one  1.090917 -0.302119
b
  key1 key2     data1     data2
2    b  one -0.382953 -0.669161
3    b  two -1.619178 -0.782035


Trong trường hợp có nhiều key, phần tử đầu tiên của tupple sẽ là 1 tupple của các key:

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
  print((k1, k2))
  print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.481654 -1.107212
4    a  one  1.090917 -0.302119
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.28584  1.835409
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.382953 -0.669161
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.619178 -0.782035


Tất nhiên bạn có thể chọn làm bất cứ điều gì bạn muốn với các mảng dữ liệu. 1 gợi ý hữu ích dành cho bạn là thực hiện tính toán dựa trên 1 dict của các mảng dữ liệu bằng 1 bước phụ:

In [ ]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one  0.481654 -1.107212
  1    a  two -0.285840  1.835409
  4    a  one  1.090917 -0.302119), ('b',   key1 key2     data1     data2
  2    b  one -0.382953 -0.669161
  3    b  two -1.619178 -0.782035)]

In [ ]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.382953,-0.669161
3,b,two,-1.619178,-0.782035


Mặc định `groupby` thực hiện group dựa vào `axis=0`, nhưng bạn cũng có thể thực hiện group với `axis=1`.

In [ ]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

Chúng ta cũng có thể print các group:

In [ ]:
for dtype, group in grouped:
  print(dtype)
  print(group)

a
0    0.481654
1   -0.285840
4    1.090917
Name: data1, dtype: float64
b
2   -0.382953
3   -1.619178
Name: data1, dtype: float64


### Selecting a Column or Subset of Columns

Index 1 đối tượng GroupBy được tạo bởi DataFrame và 1 column name hoặc 1 array các column name có kết quả tương tự việc subset dữ liệu để tổng hợp. Vì vậy cách viết dưới đây:

In [ ]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

sẽ tương đương với:

In [ ]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

Đặc biệt hơn đối với các dataset lớn, có thể chỉ cần tổng hợp 1 vài cột cụ thể. Ví dụ trong dataset trước, để tính toán mean chỉ cho cột `data2` và nhận kết quả là 1 DataFrame, ta có thể viết:

In [ ]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.704666
     two   1.835409
b    one  -0.669161
     two  -0.782035

Đối tượng được trả về bởi thao thác index là một grouped DataFrame nếu được truyền vào 1 list hoặc array, trả về 1 Series nếu chỉ các column name được truyền vào.

In [ ]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [ ]:
s_grouped.mean()

key1  key2
a     one    -0.704666
      two     1.835409
b     one    -0.669161
      two    -0.782035
Name: data2, dtype: float64

### Grouping with Dicts and Series

Thông tin sử dụng để thực hiện thao tác group có thể tồn tại không chỉ dưới dạng array. Cùng xem ví dụ sau:

In [ ]:
import numpy as np
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.510578,-0.141044,-0.386188,-0.947080,0.190947
Steve,0.198324,0.400328,-1.270892,0.417402,-0.766241
Wes,-0.334757,NaN,NaN,-0.533153,-0.619028
Jim,0.007396,0.078962,1.829642,-1.425174,1.671486
Travis,1.214033,-0.714027,1.968942,0.125382,-0.791088


Giả sử tôi có 1 nhóm các cột và muốn thực hiện phép `sum()` đối với các cột trong mỗi group:

In [ ]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [ ]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.333268,-0.460676
Steve,-0.853490,-0.167588
Wes,-0.533153,-0.953786
Jim,0.404468,1.757843
Travis,2.094324,-0.291082


In [ ]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [ ]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

Sử dụng Python function là 1 cách đơn giản hơn để định nghĩa 1 group mapping so với dùng dict hay Series. Bất kỳ function nào được truyền vào dưới dạng group key sẽ được gọi mỗi lần cho 1 giá trị index, với giá trị trả về được dùng như là tên group. Cụ thể hơn, cùng xem xét DataFrame ở ví dụ trước có first name được sử dụng làm giá trị index. Giả sử bạn muốn nhóm dữ liệu theo độ dài của tên, trong khi bạn có thể tạo ra 1 array chứa độ dài của tên thì truyền vào groupby method bằng 'len' function trở nên đơn giản hơn nhiều:

In [ ]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.837939,-0.062083,1.443454,-2.905406,1.243404
5,0.198324,0.400328,-1.270892,0.417402,-0.766241
6,1.214033,-0.714027,1.968942,0.125382,-0.791088


Sử dụng lẫn lộn array, dict hay Series không phải là vấn đề bởi tất cả chúng đều sẽ được ngầm chuyển đổi thành array:

In [ ]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [ ]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.510578 -0.141044 -0.386188 -0.947080 -0.619028
  two  0.007396  0.078962  1.829642 -1.425174  1.671486
5 one  0.198324  0.400328 -1.270892  0.417402 -0.766241
6 two  1.214033 -0.714027  1.968942  0.125382 -0.791088

### Grouping by Index Levels

Tiện ích cuối cùng của dataset có index phân cấp là khả năng tổng hợp bằng cách sử dụng 1 trong các index level. Cùng xem 1 ví dụ:

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.483667  1.518591 -1.505168  0.944122  1.254277
1      0.716802 -1.601692 -0.963396  1.423921  0.861445
2     -1.785945  0.038255  0.109702 -0.298601  0.247822
3      0.244951 -0.108834  0.501629  0.092862  1.569336

Để group theo level, truyền vào số chỉ level đó hoặc tên cho tham số `level`:

In [ ]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 8.2 Data Aggregation

Aggregation (phép tổng hợp) đề cập đến bất kỳ phép biến đổi dữ liệu nào tạo ra các giá trị vô hướng (scalar value) từ các array. Các ví dụ chúng ta đã gặp sử dụng một trong số chúng, bao gồm mean, count, min, và sum. Bạn có thể tự hỏi điều gì đang xảy ra khi bạn gọi hàm mean() trên một GroupBy object. Các phép tổng hợp phổ biến được liệt kê trong bảng dưới đây:

| Function name | Description                                                  |
| :------------ | :----------------------------------------------------------- |
| `count`       | Number of non-NA values in the group                         |
| `sum`         | Sum of non-NA values                                         |
| `mean`        | Mean of non-NA values                                        |
| `median`      | Arithmetic median of non-NA values                           |
| `std, var`    | Unbiased (n – 1 denominator) standard deviation and variance |
| `min, max`    | Minimum and maximum of non-NA values                         |
| `prod`        | Product of non-NA values                                     |
| `first, last` | First and last non-NA values                                 |

Bạn có thể sử dụng các phép tổng hợp do bạn phát triển và sử dụng thêm bất kỳ method nào được áp dụng trên grouped object. Ví dụ bạn có thể nhớ lại rằng quantile tính toán quantile mẫu của 1 Series hay 1 cột trong DataFrame.

Trong khi quantile không được thực hiện 1 cách rõ ràng bằng GroupBy method, nó là 1 Series method do đó vẫn sẵn sàng được sử dụng. Ngầm định, GroupBy thực hiện slice trên series, gọi `piece.quantile(0.9) cho mỗi phần, sau đó ghép những kết quả này lại với nhau thành kết quả cuối cùng:

In [ ]:
df

,key1,key2,data1,data2
0,a,one,0.548810,0.444728
1,a,two,1.950637,-1.690023
2,b,one,-0.272943,-1.925146
3,b,two,-0.308027,0.921635
4,a,one,1.720689,-1.169202


In [ ]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.904647
b   -0.276451
Name: data1, dtype: float64

Để sử dụng phép tổng hợp bằng function của riêng bạn, truyền vào bất kỳ function nào có thể tổng hợp 1 array thành `aggregate` hoặc `agg` method:

In [ ]:
def peak_to_peak(arr):
  return arr.max() - arr.min()

grouped.agg(peak_to_peak)

total_bill   tip  size   tip_pct
day  smoker                                  
Fri  No           10.29  2.00     1  0.067349
     Yes          34.42  3.73     3  0.159925
Sat  No           41.08  8.00     3  0.235193
     Yes          47.74  9.00     4  0.290095
Sun  No           39.40  4.99     4  0.193226
     Yes          38.10  5.00     3  0.644685
Thur No           33.68  5.45     5  0.193350
     Yes          32.77  3.00     2  0.151240

Bạn có thể để ý rằng, một số method như `describe` cũng có thể hoạt động tốt, mặc dù chúng không phải là các method chuyên dùng để tổng hợp:

In [ ]:
grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0  1.406712  0.751808  ... -1.169202 -0.362237  0.444728
b      2.0 -0.290485  0.024808  ... -0.501755  0.209940  0.921635

[2 rows x 16 columns]

> Lưu ý: Các custom function thường sẽ chậm hơn so với các function được tối ưu cho phép tổng hợp như đã đề cập trong bảng ở phần nội dung trên.

### Column-Wise and Multiple Function Application | Tổng hợp dữ liệu theo cột & áp dụng nhiều hàm tính toán cùng lúc

In [ ]:
import pandas as pd
tips = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/demo/tips_dataset.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


Như bạn đã thấy, tổng hợp 1 Series hay các cột của 1 DataFrame là vấn đề sử dụng các method mong muốn hoặc dùng các method như mean hoặc std. Tuy nhiên bạn có thể muốn tổng hợp bằng 1 hàm khác tùy thuộc vào cột hoặc nhiều hàm cùng một lúc. May thay, điều này hoàn toàn có thể thực hiện được. Hãy cùng tham khảo một số ví dụ dưới đây, chúng ta sẽ nhóm thông tin tips theo day và smoker:

In [ ]:
grouped = tips.groupby(['day', 'smoker'])

Với các hàm đã nhắc đến ở bảng phía trên như mean, sum bạn có thể truyền tên của chúng dưới dạng 1 string khi thực hiện group:

In [ ]:
grouped_pct = grouped['tip_pct']

In [ ]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

Nếu bạn truyền vào list các function hoặc chỉ mình tên function, bạn sẽ nhận về 1 DataFrame với tên columns được lấy từ tên của function:

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Ở đây chúng ta truyền vào 1 list các function tổng hợp cho `agg` method tính toán trên các nhóm dữ liệu.

Bạn có thể đặt tên tùy chọn cho các cột mà không nhất thiết phải giữ nguyên tên mặc định tạo bởi các function, đáng chú ý là các lambda function có tên là lamdbda khiên cho chúng khó có thể phân biệt. Vì vậy, nếu bạn truyền vào 1 list các tuple có dạng (name, function) thì phần tử đầu tiên của mỗi tuple sẽ được dùng để đặt tên cho các cột của DataFrame.

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Với DataFrame, bạn có nhiều tùy chọn hơn, bạn có thể cung cấp 1 list các function để áp dụng hàng loạt trên các cột hoặc mỗi function áp dụng trên 1 cột riêng biệt. Giả sử, chúng ta muốn tính toán count, mean và mã cho cột `tip_pct` và `total_bill`:

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

Bạn thấy đấy, DataFrame trả về có các cột được xếp chồng (hierachical columns), giống như bạn thực hiện tổng hợp từng cột riêng biệt và sử dụng concat method để merge các kết quả lại với nhau:

In [ ]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

Như đã đề cập, 1 list các tuple với custom name do bạn đặt cũng có thể được sử dụng trong trường hợp này:

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Giờ đây, giả sử bạn muốn sử dụng các hàm riêng biệt cho 1 hoặc nhiều cột. Để thực hiện điều này, hãy truyền 1 dict vào `agg` method bao gồm mapping của tên cột và function hoặc list function tương ứng:

In [ ]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [ ]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

> Một DataFrame sẽ hiển thị dạng cột phân cấp nếu nhiều function được áp dụng cho ít nhất 1 cột khi thực hiện phép tổng hợp.

### Returning Aggregated Data Without Row Indexes | Tổng hợp dữ liệu, không kèm Index theo hàng

Trong các ví dụ từ đầu bài này tới giờ, các dữ liệu tổng hợp trả về đều có index, có thể có phân cấp, được tạo từ việc kết hợp các các group key. Không phải lúc nào đây cũng là điều bạn muốn nhận được, do đó bạn có thể vô hiệu hóa cài đặt mặc định này bằng cách truyền vào tùy chọn `as_index=False` đối với `groupby` method:

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Tất nhiên, bạn cũng có thể đạt được kết quả tương tự bằng cách gọi `reset_index()` method nhưng việc dùng `as_index=False` giúp bạn lược bỏ đi 1 thao tác không cần thiết.

## 8.3 Apply: General split-apply-combine

Method chung nhất của thao tác GroupBy là `apply`, nó là chủ đề của phần còn lại trong bài học này. Như đã minh họa ở phần đầu, apply thực hiện tách các đối tượng đang được thao tác thành nhiều nhóm, gọi hàm để thực hiện tính toán trên mỗi nhóm sau đó ghép các nhóm này lại với nhau.

Quay trở lại tipping dataset ở trên, giả sử bạn muốn lựa chọn top 5 giá trị `tip_pct` theo nhóm. Đầu tiên, viết 1 function giúp lựa chọn các dòng dữ liệu với giá trị lớn nhất trong 1 số cột nhất định:

In [ ]:
def top(df, n, column):
  return df.sort_values(by=column)[-n:]

top(tips, n=6, column='tip_pct')

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


Giờ đây, nếu chúng ta thực hiện group theo cột `smoker`, sau đó gọi `apply` method với function này chúng ta sẽ có kết quả như sau:

In [ ]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

Điều gì đã xảy ra ở đây? `top` function được gọi trên mỗi nhóm các hàng từ DataFrame, sau đó kết quả được gắn với nhau bằng cách sử dụng pandas.concat, cuối cùng gắn label bằng tên nhóm cho các mảng dữ liệu. Do đó, kết quả có index phân cấp và các index cấp thấp thì chứa các giá trị index từ DataFrame gốc.

Nếu bạn truyền 1 function để apply và có các đối số hoặc tham số, bạn có thể đặt các thông tin này sau function như minh họa dưới đây:

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size   tip_pct
smoker day                                                                     
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4  0.115982

> Lưu ý: Ngoài các kỹ thuật sử dụng cơ bản này, để tận dụng tối đa `apply` yêu cầu sự sáng tạo từ bạn. Điều gì xảy ra bên trong function được truyền vào là tùy thuộc vào bạn, nó chỉ trả về 1 pandas object hoặc 1 giá trị vô hướng. Phần còn lại của bài học sẽ tập trung vào các ví dụ cho bạn thấy cách giải quyết các vấn đề khác nhau với `groupby`.

Bạn có thể nhớ rằng chúng ta đã gọi `describe` trước đó với GroupBy object:

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [ ]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Thực chất bên trong GroupBy method, khi bạn gọi 1 method chẳng hạn describe, nó chẳng qua là a shortcut cho:

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

Trong ví dụ trước đó, bạn thấy rằng, đối tượng kết quả có index phân cấp dưới dạng group key cùng với index của các mảnh dữ liệu thuộc đối tượng ban đầu. Bạn có thể thay đổi hành vi này bằng cách truyền tham số `group_keys=False` vào groupby method:

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

Bạn hẳn còn nhớ bài học 6.2.5 về Tách & phân nhóm dữ liệu, Pandas có một số công cụ, đặc biệt là `cut` và `qcut` để slice dữ liệu thành các nhóm phân loại tùy chỉnh bằng cách lấy mẫu phân phối. Kết hợp function này với `groupby` khiến nó trở nên cực kỳ tiện lợi để thực hiện phân tích phân phối (quantile analysis) trên dataset.


In [ ]:
import numpy as np
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame['data1'], 4)
quartiles[:10]

0      (-0.242, 1.43]
1    (-1.915, -0.242]
2    (-1.915, -0.242]
3    (-1.915, -0.242]
4    (-1.915, -0.242]
5    (-3.594, -1.915]
6      (-0.242, 1.43]
7      (-0.242, 1.43]
8      (-0.242, 1.43]
9      (-0.242, 1.43]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.594, -1.915] < (-1.915, -0.242] < (-0.242, 1.43] <
                                    (1.43, 3.103]]

Đối tượng danh mục được trả về bởi `cut` có thể được truyền trực tiếp tới `groupby`. Vì vậy chúng ta có thể tính toán 1 nhóm các chỉ số thống kê cho cột `data2` như sau:

In [ ]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.594, -1.915]",-1.696326,1.781018,27.0,-0.059291
"(-1.915, -0.242]",-2.970740,3.953968,397.0,-0.050607
"(-0.242, 1.43]",-2.988641,3.357352,504.0,-0.052422
"(1.43, 3.103]",-2.622567,2.475158,72.0,-0.193498


In [ ]:
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.594, -1.915]",-1.696326,1.781018,27.0,-0.059291
"(-1.915, -0.242]",-2.970740,3.953968,397.0,-0.050607
"(-0.242, 1.43]",-2.988641,3.357352,504.0,-0.052422
"(1.43, 3.103]",-2.622567,2.475158,72.0,-0.193498


Đây là các nhóm có số phần tử bằng nhau, để tính toán các nhóm có phân phối bằng nhau (số phần tử có thể khác nhau) hãy sử dụng qcut. Thử truyền vào `labels=False` vào để lấy chỉ mình các mốc phân phối:

In [ ]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.448905,1.781018,100.0,-0.086376
1,-1.973197,2.606698,100.0,-0.027420
2,-2.970740,2.319003,100.0,-0.014704
3,-2.931401,2.171525,100.0,-0.164001
4,-2.249372,3.953968,100.0,0.089024
5,-2.595156,2.230062,100.0,0.104014
6,-2.988641,1.685107,100.0,-0.105129
7,-2.545373,3.357352,100.0,-0.087900
8,-2.441230,2.819309,100.0,-0.163958


### Example: Filling Missing Values with Group-Specific Values

Khi xử lý missing data, trong một số trường hợp bạn sẽ loại bỏ dữ liệu quan sát được bằng `dropna` method, lúc khác bạn có thể muốn điền các dữ liệu khuyết này sử dụng 1 giá trị cố định hoặc 1 số giá trị bắt nguồn từ dataset. `fillna` là công cụ sử dụng phù hợp, ở đây chúng ta sẽ thử fill các giá trị NA bởi giá trị mean:

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.772477
2         NaN
3    1.541987
4         NaN
5   -1.387362
dtype: float64

In [ ]:
s.fillna(s.mean())

0   -0.205951
1   -0.772477
2   -0.205951
3    1.541987
4   -0.205951
5   -1.387362
dtype: float64

Giả sử bạn cần giá trị muốn điền thay đổi theo nhóm. Một cách để thực hiện điều này là `group` dữ liệu và sử dụng `apply` function để gọi `fillna` trên mỗi đoạn dữ liệu. Dưới đây là dữ liệu mẫu về các bang của Hoa Kỳ được chia thành các khu vực phía đông và tây:

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          1.281606
New York     -1.184426
Vermont      -0.692153
Florida       1.805247
Oregon        0.069897
Nevada       -1.621054
California    1.836662
Idaho         1.609715
dtype: float64

Lưu ý rằng cú pháp `['East'] * 4` cho ra 1 list bao gồm 4 bản sao của `['East']`.

Hãy đặt 1 số giá trị của dataset như là giá trị missing:

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.281606
New York     -1.184426
Vermont            NaN
Florida       1.805247
Oregon        0.069897
Nevada             NaN
California    1.836662
Idaho              NaN
dtype: float64

In [ ]:
data.groupby(group_key).mean()

East    0.634142
West    0.953279
dtype: float64

Chúng ta có thể fill các giá trị NA bằn group mean như sau:

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.281606
New York     -1.184426
Vermont       0.634142
Florida       1.805247
Oregon        0.069897
Nevada        0.953279
California    1.836662
Idaho         0.953279
dtype: float64

Trong trường hợp khác, bạn có thể tự định nghĩa giá trị cần fill ngay trong code và giá trị này thay đổi theo từng nhóm. Bởi vì các group có sẵn attribute nội tại, chúng ta có thể sử dụng cú pháp:

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          1.281606
New York     -1.184426
Vermont       0.500000
Florida       1.805247
Oregon        0.069897
Nevada       -1.000000
California    1.836662
Idaho        -1.000000
dtype: float64

## 8.4. Pivot Table & Cross-Tabulation

Pivot table là 1 công cụ tóm tắt dữ liệu thường thấy trong các chương trình bảng tính (spreadsheet) và phần mềm phân tích dữ liệu/Nó giúp tổng hợp một bảng dữ liệu bằng một hoặc nhiều key, sắp xếp dữ liệu trong một hình chữ nhật với một số group key theo hàng và một số key theo cột. Pivot table trong Python với Pandas được thực hiện thông qua cơ sở sử dụng GroupBy kết hợp với các thao tác reshape sử dụng index phân cấp.

DataFrame có `pivot_table` method và cũng có hàm `pandas.pivot`. Ngoài việc giúp cho việc thực hiện thao tác GroupBy được thuận tiện hơn thì Pivot table có thể tính thêm tổng từng phần được gọi là margins.

### Pivot Table Explained

#### Pivot Table Syntax

Trước tiên cùng xem qua cấu trúc của hàm pivot_table trong Python. Tài liệu chính thức có thể tìm thấy ` [Tại đây](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html).

In [ ]:
import pandas as pd
pd.pivot_table(data, index=None, values=None,  columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)

Đây là phần mô tả các tùy chọn của pivot_table function:

| Function name | Description                                                  |
| :------------ | :----------------------------------------------------------- |
| `values`      | Column name or names to aggregate; by default aggregates all numeric columns |
| `index`       | Column names or other group keys to group on the rows of the resulting pivot table |
| `columns`     | Column names or other group keys to group on the columns of the resulting pivot table |
| `aggfunc`     | Aggregation function or list of functions (`'mean'` by default); can be any function valid in a groupby context |
| `fill_value`  | Replace missing values in result table                       |
| `dropna`      | If `True`, do not include columns whose entries are all `NA` |
| `margins`     | Add row/column subtotals and grand total (`False` by default) |
| margins_name  | Name of total row / column (default = 'All')                 |
| observed      | Only for categorical data – if True will only show observed values for categorical groups (default = False) |

Cùng tham khảo sample dataset để minh họa cho các ví dụ tiếp theo:

In [ ]:
import pandas as pd
df = pd.read_excel('https://github.com/datagy/pivot_table_pandas/raw/master/sample_pivot.xlsx', parse_dates=['Date'])
df.head()

,Date,Region,Type,Units,Sales
0,2020-07-11,East,Children's Clothing,18.0,306
1,2020-09-23,North,Children's Clothing,14.0,448
2,2020-04-02,South,Women's Clothing,17.0,425
3,2020-02-28,East,Children's Clothing,26.0,832
4,2020-03-19,West,Women's Clothing,3.0,33


#### Creating a Pivot Table in Pandas

Để tạo 1 pivot table đơn giản nhất, bạn cần truyền vào DataFrame cần tổng hợp dữ liệu, cùng tham số `index` và `values`.

In [ ]:
sales_by_region = pd.pivot_table(df, index = 'Region', values = 'Sales')
sales_by_region

,Sales
Region,
East,408.182482
North,438.924051
South,432.956204
West,452.029412


Mặc định pivot_table function sẽ tổng hợp dữ liệu theo `mean` function của các giá trị khi bạn không chỉ rõ tùy chọn `aggfunc`. Hãy thử sử dụng `aggfunc='sum'` để tính tổng thay vì tính giá trị trung bình như trên:

In [ ]:
total_by_region = pd.pivot_table(df, index = 'Region', values = 'Sales', aggfunc='sum')
total_by_region

,Sales
Region,
East,167763
North,138700
South,59315
West,61476


Như các bạn thấy, giá trị aggfunc có thể truyền vào là 1 string chứa tên của hàm bạn muốn tổng hợp. Bạn có thể sử dụng rất nhiều hàm đa dạng như các hàm tính toán, thống kê của Pandas, Numpy, vv.

Bảng dưới đây thể hiện danh sách các hàm tính toán cơ bản của Pandas:

|  #   | Function |           Description            |
| :--: | :------: | :------------------------------: |
|  1   |  count   | Number of non-null observations  |
|  2   |   sum    |          Sum of values           |
|  3   |   mean   |          Mean of Values          |
|  4   |  median  |         Median of Values         |
|  5   |   mode   |          Mode of values          |
|  6   |   std    | Standard Deviation of the Values |
|  7   |   min    |          Minimum Value           |
|  8   |   max    |          Maximum Value           |
|  9   |   abs    |          Absolute Value          |
|  10  |   prod   |        Product of Values         |
|  11  |  cumsum  |          Cumulative Sum          |
|  12  | cumprod  |        Cumulative Product        |
| 13 | pd.Series.nunique | Unique Count|

Bạn cũng có thể sử dụng hàm của Pandas Series ví dụ nunique (đếm giá trị unique) và nhiều hàm khác [Tại đây](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#computations-descriptive-stats). Ngoài ra tương tự với `groupby`, bạn có thể tự định nghĩa function để tổng hợp dữ liệu với pivot_table nếu cần thiết.

In [ ]:
pd.pivot_table(df, index = 'Region', values = 'Type', aggfunc=pd.Series.nunique)

,Type
Region,
East,3
North,3
South,3
West,3


Như các bạn thấy đấy, `pivot_table` trả về kết quả là các đối tượng chứa index phân cấp, để chuyển chúng thành DataFrame bình thường, bạn có thể áp dụng `reset_index()` function như dưới đây:

In [ ]:
sales_by_region.reset_index()

,Region,Sales
0,East,408.182482
1,North,438.924051
2,South,432.956204
3,West,452.029412


#### Filtering Python Pivot Tables

Bạn đã học được cách tạo ra các pivot table đơn giản, giả sử bạn muốn filter một số dữ liệu nhất định để tạo pivot table thì sao nhỉ? Cùng xem qua ví dụ minh họa với DataFrame với mean Sale và Region:

In [ ]:
avg_region_price = pd.pivot_table(df, index = 'Region', values = 'Sales')
avg_region_price

,Sales
Region,
East,408.182482
North,438.924051
South,432.956204
West,452.029412


Giả sử bạn chỉ muốn filter DataFrame để lấy chỉ các giá trị `Regions` có sale price lớn hơn 450:

In [ ]:
avg_region_price[avg_region_price['Sales'] > 450]

,Sales
Region,
West,452.029412


Bạn cũng có thể áp dụng nhiều điều kiện filter cùng lúc, ví dụ filter chỉ những giá trị sales lớn hơn 450 hoặc nhỏ hơn 430:

In [ ]:
avg_region_price[(avg_region_price['Sales'] > 450) | (avg_region_price['Sales'] < 430)]

,Sales
Region,
East,408.182482
West,452.029412


> Lưu ý rằng chúng ta gói gọn mỗi điều kiện filter bằng cặp dấu ngoặc đơn `()` và ngăn cách các điều kiện khác nhau (trong điều kiện filter là OR) bởi ký tự `|` gọi là `pipe`.

#### Multi-index pivot table

Nếu bạn muốn có 1 pivot table với index phân cấp, đơn giản là truyền vào 1 list tên các cột các bạn muốn group:

In [ ]:
multi_index = pd.pivot_table(df, index = ['Region', 'Type'], values = 'Sales', aggfunc = 'sum')
multi_index

Sales
Region Type                      
East   Children's Clothing  45849
       Men's Clothing       51685
       Women's Clothing     70229
North  Children's Clothing  37306
       Men's Clothing       39975
       Women's Clothing     61419
South  Children's Clothing  18570
       Men's Clothing       18542
       Women's Clothing     22203
West   Children's Clothing  20182
       Men's Clothing       19077
       Women's Clothing     22217

#### Find the relationship between features with columns parameter

Thêm các cột vào pivot table trong Pandas có thể thêm nhiều chiều thông tin để phân tích. Như cú pháp của pivot_table đã đề cập, bạn có thể thực hiện điều này bằng cách truyền tên cột cần thêm vào tham số `columns=` như dưới đây:

In [ ]:
columns_example = pd.pivot_table(df, index = 'Type', columns = 'Region', values = 'Units', aggfunc = 'sum')
columns_example

Region,East,North,South,West
Type,,,,
Children's Clothing,2318.0,1763.0,1017.0,789.0
Men's Clothing,2420.0,0.0,725.0,829.0
Women's Clothing,3372.0,2596.0,1056.0,1006.0


Bằng cách tổng hợp dữ liệu như này, chúng ta có thể dễ dàng phân tích và so sánh hiệu quả bán hàng theo các khu vực.

#### Handling missing data

Không có dataset nào là hoàn hảo cả. Khi trong dataset chứa missing data và bạn thực hiện tạo pivot table sẽ trả về 1 đối tượng cũng chứa missing data. Trong một số trường hợp bạn sẽ không muốn nhìn tháy giá trị `NA` thay vào đó có thể thay thế bằng giá trị `0`.

In [ ]:
pd.pivot_table(df, index = 'Type', values = 'Units', columns = 'Region', aggfunc = 'max')

Region,East,North,South,West
Type,,,,
Children's Clothing,35.0,35.0,35.0,34.0
Men's Clothing,35.0,NaN,34.0,35.0
Women's Clothing,35.0,35.0,35.0,33.0


Để thay thế giá trị missing hiển thị trong pivot table, bạn truyền vào giá trị cần thay thế với tham số `fill_value=`:

In [ ]:
pd.pivot_table(df, index = 'Type', values = 'Units', columns = 'Region', aggfunc = 'max', fill_value = '0')

Region,East,North,South,West
Type,,,,
Children's Clothing,35.0,35,35.0,34.0
Men's Clothing,35.0,0,34.0,35.0
Women's Clothing,35.0,35,35.0,33.0


#### Adding Totals for Rows and Columns to Pandas Pivot Tables

Như các bạn để ý, các pivot table ví dụ trên đây không chứa giá trị Grand Total như các bạn thường thấy khi làm việc với Excel, để hiển thị giá trị total này bạn sử dụng tham số `margins=True`, để đặt tên cho dòng này, bạn sử dụng tham số `margins_name=`:

In [ ]:
pd.pivot_table(df, index = 'Type', values = 'Units', columns = 'Region', aggfunc = 'sum', fill_value = 0, margins = True, margins_name='Grand Total')

Region,East,North,South,West,Grand Total
Type,,,,,
Children's Clothing,35,35,35,34,35.0
Men's Clothing,35,0,34,35,35.0
Women's Clothing,35,35,35,33,35.0
Grand Total,35,35,35,35,35.0


#### Pivot Table with multiple Aggregating Function

Trong nhiều trường hợp bạn có thể sẽ muốn áp dụng nhiều hàm tổng hợp (aggregating function) cho mỗi nhóm kết quả, đơn giản là truyền tên các hàm bạn muốn sử dụng vào tham số `aggfunc=` dưới dạng 1 list:

In [ ]:
multi_functions = pd.pivot_table(df, index = ['Region', 'Type'], values = 'Sales', aggfunc = ['sum', 'mean'])
multi_functions

sum        mean
                            Sales       Sales
Region Type                                  
East   Children's Clothing  45849  405.743363
       Men's Clothing       51685  423.647541
       Women's Clothing     70229  399.028409
North  Children's Clothing  37306  438.894118
       Men's Clothing       39975  449.157303
       Women's Clothing     61419  432.528169
South  Children's Clothing  18570  412.666667
       Men's Clothing       18542  475.435897
       Women's Clothing     22203  418.924528
West   Children's Clothing  20182  480.523810
       Men's Clothing       19077  465.292683
       Women's Clothing     22217  419.188679

#### Different aggregation function for different features

Đôi khi bạn muốn áp dụng mỗi hàm tổng hợp riêng biệt cho mỗi cột, hãy truyền vào tham số `aggfunc=` với 1 dict chỉ rõ cột nào sử dụng hàm tổng hợp nào:

In [ ]:
multi_functions = pd.pivot_table(df, index = ['Region', 'Type'], values = ['Sales', 'Units'], aggfunc = {'Sales':'sum', 'Units':'mean'})
multi_functions

Sales      Units
Region Type                                 
East   Children's Clothing  45849  20.513274
       Men's Clothing       51685  19.836066
       Women's Clothing     70229  19.159091
North  Children's Clothing  37306  20.741176
       Men's Clothing       39975        NaN
       Women's Clothing     61419  18.281690
South  Children's Clothing  18570  22.600000
       Men's Clothing       18542  18.589744
       Women's Clothing     22203  19.924528
West   Children's Clothing  20182  18.785714
       Men's Clothing       19077  20.219512
       Women's Clothing     22217  18.981132

Có thể bạn sẽ muốn đổi tên `Sales` thành `Total Sale` và `Units` thành `Avg. Units` để thể hiện rõ phân tích của mình. Hãy sử dụng `reset_index()` method và rename lại tên các cột này:

In [ ]:
multi_functions = multi_functions.reset_index().rename(columns={'Sales':'Total Sales', 'Units':'Avg. Units'})
multi_functions

,Region,Type,Total Sales,Avg. Units
0,East,Children's Clothing,45849,20.513274
1,East,Men's Clothing,51685,19.836066
2,East,Women's Clothing,70229,19.159091
3,North,Children's Clothing,37306,20.741176
4,North,Men's Clothing,39975,NaN
5,North,Women's Clothing,61419,18.281690
6,South,Children's Clothing,18570,22.600000
7,South,Men's Clothing,18542,18.589744
8,South,Women's Clothing,22203,19.924528
9,West,Children's Clothing,20182,18.785714


> Lưu ý: pivot_table tự động loại bỏ missing value của index khi tổng hợp dữ liệu. Có thể bạn sẽ muốn sử dụng `fillna` method để điền các giá trị bị thiếu trong dataset trước khi thực hiện pivot.

### Cross-Tabulations: Crosstab

Một cross-tabulation (bảng chéo hay crosstab) là 1 trường hợp đặc biệt của pivot table cho phép tính toán tần xuất của nhóm. Ví dụ:

In [ ]:
demo_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_marketing_campaign/combine_sample_marketing_campaign.csv')
demo_df.head(3)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0


Trong 1 vài phân tích, có thể bạn muốn tổng hợp dữ liệu theo `Marital_Status` và `Education`. Bạn có thế sử dụng `pivot_table` tuy nhiên trường hợp này `pandas.crosstab` tỏ ra tiện lợi hơn:

In [ ]:
pd.crosstab(demo_df['Education'], demo_df['Marital_Status'], margins=True)

Marital_Status,Alone,Divorced,Married,Single,Together,Widow,All
Education,,,,,,,
2n Cycle,0,20,72,36,53,4,185
Basic,0,1,15,15,12,0,43
Graduation,1,104,377,225,255,30,992
Master,1,32,124,66,93,11,327
PhD,1,49,179,94,108,22,453
All,3,206,767,436,521,67,2000


Hai tham số đầu tiên của `crosstab` có thể là array hoặc Series hoặc 1 list các array.